In [1]:
%matplotlib notebook

import h5py
import matplotlib.pyplot as plt
from numpy import *
import pandas as pd
import os
import sys
import scipy.optimize as opt
from typing import Dict, Any
import recordclass.recordclass as rc

In [2]:
def get_ivars(iteration: h5py._hl.group.Group, *ivar_names: str) -> Dict[str, Any]:
    """
    Gets the values of independent variables for the iteration that was passed in
    
    Args:
        iteration: the data group corresponding to an iteration
        *ivar_names: list of names of independent variables
    Returns:
        ivar_vals : map of ivar_names to their values
    """
    return {name: iteration[f"variables/{name}"][()] for name in ivar_names}

In [3]:
h5file = h5py.File('results.hdf5',mode='r+')

#Find number of measurements per experiment
measurements = h5file['settings/experiment/measurementsPerIteration'][()]+1
num_its = len(h5file['iterations'])

#Find independent variable names and values
indep_vars = {}
if len(h5file['iterations'])> 1:
    for variable in h5file['settings/experiment/independentVariables'].items():
        values = eval(variable[1]['function'][()])
        #print(f"{variable[0]}\n\t{values}")
        if iterable(values):
            indep_vars.update({variable[0]: list(values)})
iVars = list(indep_vars.keys())

# map iterations to independent variable values
iterations = pd.DataFrame(columns=["iteration"]+list(indep_vars.keys()))
for iteration in h5file['iterations'].items():
    ivar_vals = get_ivars(iteration[1],*list(indep_vars.keys()))
    ivar_vals.update({"iteration": int(iteration[0])})
    iterations = iterations.append(pd.DataFrame(ivar_vals, index = [iteration[0]]))

iterations = iterations.sort_values("iteration")

In [5]:
help(iterable)

Help on function iterable in module numpy:

iterable(y)
    Check whether or not an object can be iterated over.
    
    Parameters
    ----------
    y : object
      Input object.
    
    Returns
    -------
    b : bool
      Return ``True`` if the object has an iterator method or is a
      sequence and ``False`` otherwise.
    
    
    Examples
    --------
    >>> np.iterable([1, 2, 3])
    True
    >>> np.iterable(2)
    False



# Hamamatsu

In [4]:
from HamamatsuH5 import HMROI, set_frame_grabber_region
import HamamatsuH5
# Set ROI
fg = set_frame_grabber_region(h5file)
width = fg['right']-fg['left']
height = fg['bottom']-fg['top']

# For documentation try > help(HMROI)
roi = HMROI(width,height,dic = {
    "left" : 2,
    "right" : 5,
    "top" : 4,
    "bottom" : 7
})

#Load data into memory
shots_per_measurement = int(h5file['/settings/experiment/LabView/camera/shotsPerMeasurement/function'][()])
# pixel-by-pixel data indexed : [iteration, measurement, shot, y-pixel, x-pixel]
hm_pix = HamamatsuH5.load_data(h5file,roi)

#take pixel-by-pixel data and treat it into counter data then mean data
# "count" data indexed : [iteration, measurement, shot]
hm_counts = hm_pix.sum(3).sum(3)
# mean count data indexed : [iteration, shot]
mus = hm_counts.mean(1)
# standard deviation in mean indexed : [iteration, shot]
ers = hm_counts.std(1)/sqrt(measurements)

## Plot Histograms of count data

In [ ]:
fig,axarr = plt.subplots(len(iterations),1,figsize=(6,4*len(iterations)))
for iteration, row in iterations.iterrows():
    iteration = int(iteration)
    for shot in range(shots_per_measurement):
        axarr[iteration].hist(hm_counts[iteration,:,shot],histtype='step', label = f"shot {shot}",bins=30)
    axarr[iteration].legend()
    axarr[iteration].set_title(str(row))
fig.tight_layout()
fig.show()

## Plot mean count data with error bars

In [ ]:
if len(indep_vars) == 0:
    for shot in range(shots_per_measurement):
        print(f"shot {shot} rate : {mus[0,shot]} +/- {ers[0,shot]}")
elif len(indep_vars) == 1:
    xlin = iterations[list(indep_vars.keys())[0]]
    fig,ax = plt.subplots(1,1,figsize=(6,5))
    for shot in range(shots_per_measurement):
        ax.errorbar(xlin, mus[:,shot], yerr=ers[:,shot], label=f"Shot {shot}")
    ax.legend()
    ax.set_ylabel("Counts in ROI")
    ax.set_xlabel(list(indep_vars.keys())[0])
    fig.show()
else:
    print("Many axes, look for purpose made plotting cells")

## Show images taken within ROI

In [ ]:
## show average image for one shot for each iteration
im_shot = 0 # shot to image

fig,axarr = plt.subplots(len(iterations),1,figsize=(6,4*len(iterations)))
for iteration, row in iterations.iterrows():
    i = row['iteration']
    ivs = {iVar: row[iVar] for iVar in iVars}
    axarr[i].imshow(hm_pix[i,:,im_shot,:,:].mean(0))
    axarr[i].set_title(str(row))
fig.tight_layout()
fig.show()
    
    

## Plot Histgram data of each picture for each iteration
* Very memory hungry!
* Please clear output before saving, copying or pushing if this cell was run

In [ ]:
im_width = roi.right-roi.left
im_height = roi.bottom-roi.top
for iteration, row in iterations.iterrows():
    i = row['iteration']
    fig,axarr = plt.subplots(im_height,im_width,figsize = (10,10))
    for y in range(im_height):
        for x in range(im_width):
            bns = 20
            for shot in range(shots_per_measurement):
                axarr[y,x].hist(hm_pix[i,:,shot,y,x],bins=bns,histtype='step',label = f"Shot {shot}")
            axarr[y,x].set_title(f"x = {x}, y = {y}")
            axarr[y,x].legend()
    fig.suptitle(str(row))
    fig.tight_layout()
    fig.show()

* Ensure once things get better they can get involved in Madison
* Not isolated
* on slack (check)
* check on classes/TAing